# 04 - Test and Deploy Training Pipeline to Vertex Pipelines

The purpose of this notebook is to test, deploy, and run the `TFX` pipeline on `Vertex Pipelines`. The notebook covers the following tasks:
1. Run the tests locally.
2. Run the pipeline using `Vertex Pipelines`
3. Execute the pipeline deployment `CI/CD` steps using `Cloud Build`.

## Setup

### Import libraries

In [14]:
import os
import kfp
import tfx

print("Tensorflow Version:", tfx.__version__)
print("KFP Version:", kfp.__version__)

Tensorflow Version: 1.2.0
KFP Version: 1.8.9


### Setup Google Cloud project

In [2]:
PROJECT = 'grandelli-demo-295810' # Change to your project id.
REGION = 'us-central1' # Change to your region.
BUCKET =  'grandelli-demo-295810-partner-training-2022' # Change to your bucket name.
SERVICE_ACCOUNT = "155283586619-compute@developer.gserviceaccount.com"

if PROJECT == "" or PROJECT is None or PROJECT == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT = shell_output[0]
    
if SERVICE_ACCOUNT == "" or SERVICE_ACCOUNT is None or SERVICE_ACCOUNT == "[your-service-account]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.account)' 2>/dev/null
    SERVICE_ACCOUNT = shell_output[0]
    
if BUCKET == "" or BUCKET is None or BUCKET == "[your-bucket-name]":
    # Get your bucket name to GCP project id
    BUCKET = PROJECT
    # Try to create the bucket if it doesn't exists
    ! gsutil mb -l $REGION gs://$BUCKET
    print("")
    
print("Project ID:", PROJECT)
print("Region:", REGION)
print("Bucket name:", BUCKET)
print("Service Account:", SERVICE_ACCOUNT)

Project ID: grandelli-demo-295810
Region: us-central1
Bucket name: grandelli-demo-295810-partner-training-2022
Service Account: 155283586619-compute@developer.gserviceaccount.com


### Set configurations

In [3]:
BQ_LOCATION = 'US'
BQ_DATASET_NAME = 'partner_training' # Change to your BQ dataset name.
BQ_TABLE_NAME = 'chicago_taxitrips_prep'

VERSION = 'v01'
DATASET_DISPLAY_NAME = 'chicago-taxi-tips'
MODEL_DISPLAY_NAME = f'{DATASET_DISPLAY_NAME}-classifier-{VERSION}'
PIPELINE_NAME = f'{MODEL_DISPLAY_NAME}-train-pipeline'

CICD_IMAGE_NAME = 'cicd:latest'
CICD_IMAGE_URI = f"gcr.io/{PROJECT}/{CICD_IMAGE_NAME}"

In [4]:
!rm -r src/raw_schema/.ipynb_checkpoints/

rm: cannot remove 'src/raw_schema/.ipynb_checkpoints/': No such file or directory


## 1. Run the CICD steps locally

### Set pipeline configurations for the local run

In [5]:
os.environ["DATASET_DISPLAY_NAME"] = DATASET_DISPLAY_NAME
os.environ["MODEL_DISPLAY_NAME"] =  MODEL_DISPLAY_NAME
os.environ["PIPELINE_NAME"] = PIPELINE_NAME
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION
os.environ["BQ_LOCATION"] = BQ_LOCATION
os.environ["BQ_DATASET_NAME"] = BQ_DATASET_NAME
os.environ["BQ_TABLE_NAME"] = BQ_TABLE_NAME
os.environ["GCS_LOCATION"] = f"gs://{BUCKET}/{DATASET_DISPLAY_NAME}/e2e_tests"
os.environ["TRAIN_LIMIT"] = "1000"
os.environ["TEST_LIMIT"] = "100"
os.environ["UPLOAD_MODEL"] = "0"
os.environ["ACCURACY_THRESHOLD"] = "0.1"
os.environ["BEAM_RUNNER"] = "DirectRunner"
os.environ["TRAINING_RUNNER"] = "local"

In [6]:
from src.tfx_pipelines import config
import importlib
importlib.reload(config)

for key, value in config.__dict__.items():
    if key.isupper(): print(f'{key}: {value}')

2022-03-30 14:43:03.745149: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


PROJECT: grandelli-demo-295810
REGION: us-central1
GCS_LOCATION: gs://grandelli-demo-295810-partner-training-2022/chicago-taxi-tips/e2e_tests
ARTIFACT_STORE_URI: gs://grandelli-demo-295810-partner-training-2022/chicago-taxi-tips/e2e_tests/tfx_artifacts
MODEL_REGISTRY_URI: gs://grandelli-demo-295810-partner-training-2022/chicago-taxi-tips/e2e_tests/model_registry
DATASET_DISPLAY_NAME: chicago-taxi-tips
MODEL_DISPLAY_NAME: chicago-taxi-tips-classifier-v01
PIPELINE_NAME: chicago-taxi-tips-classifier-v01-train-pipeline
ML_USE_COLUMN: ml_use
EXCLUDE_COLUMNS: trip_start_timestamp
TRAIN_LIMIT: 1000
TEST_LIMIT: 100
SERVE_LIMIT: 0
NUM_TRAIN_SPLITS: 4
NUM_EVAL_SPLITS: 1
ACCURACY_THRESHOLD: 0.1
USE_KFP_SA: False
TFX_IMAGE_URI: gcr.io/grandelli-demo-295810/tfx-chicago-taxi-tips:latest
BEAM_RUNNER: DirectRunner
BEAM_DIRECT_PIPELINE_ARGS: ['--project=grandelli-demo-295810', '--temp_location=gs://grandelli-demo-295810-partner-training-2022/chicago-taxi-tips/e2e_tests/temp']
BEAM_DATAFLOW_PIPELINE_ARG

### Run unit tests

In [7]:
!python -m pytest src/tests/datasource_utils_tests.py -s

============================= test session starts ==============================
platform linux -- Python 3.7.12, pytest-7.1.0, pluggy-1.0.0
rootdir: /home/jupyter/20220318_Training/gcp-partner-training-mlops
plugins: anyio-3.3.4
collected 2 items                                                              

src/tests/datasource_utils_tests.py BigQuery Source: grandelli-demo-295810.partner_training.chicago_taxitrips_prep
.BigQuery Source: grandelli-demo-295810.partner_training.chicago_taxitrips_prep
.

============================== 2 passed in 11.23s ==============================


In [8]:
!python -m pytest src/tests/model_tests.py -s

============================= test session starts ==============================
platform linux -- Python 3.7.12, pytest-7.1.0, pluggy-1.0.0
rootdir: /home/jupyter/20220318_Training/gcp-partner-training-mlops
plugins: anyio-3.3.4
collecting ... 2022-03-30 14:43:40.820150: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
collected 2 items                                                              

src/tests/model_tests.py .2022-03-30 14:43:43.197665: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-03-30 14:43:43.197715: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to c

### Run e2e pipeline test

In [9]:
!python -m pytest src/tests/pipeline_deployment_tests.py::test_e2e_pipeline -s

============================= test session starts ==============================
platform linux -- Python 3.7.12, pytest-7.1.0, pluggy-1.0.0
rootdir: /home/jupyter/20220318_Training/gcp-partner-training-mlops
plugins: anyio-3.3.4
collecting ... 2022-03-30 14:43:45.669774: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
collected 1 item                                                               

src/tests/pipeline_deployment_tests.py upload_model: 0
Pipeline e2e test artifacts stored in: gs://grandelli-demo-295810-partner-training-2022/chicago-taxi-tips/e2e_tests
ML metadata store is ready.
Excluding no splits because exclude_splits is not set.
Excluding no splits because exclude_splits is not set.
Pipeline components: ['HyperparamsGen', 'TrainDataGen', 'TestDataGen', 'StatisticsGen', 'SchemaImporter', 'ExampleValidator', 'DataTransformer', 'WarmstartModelResolver', 'ModelTrainer', 'BaselineModelResolver', 'ModelE

## 2. Run the training pipeline using Vertex Pipelines



### Set the pipeline configurations for the Vertex AI run

In [10]:
os.environ["DATASET_DISPLAY_NAME"] = DATASET_DISPLAY_NAME
os.environ["MODEL_DISPLAY_NAME"] = MODEL_DISPLAY_NAME
os.environ["PIPELINE_NAME"] = PIPELINE_NAME
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION
os.environ["GCS_LOCATION"] = f"gs://{BUCKET}/{DATASET_DISPLAY_NAME}"
os.environ["TRAIN_LIMIT"] = "85000"
os.environ["TEST_LIMIT"] = "15000"
os.environ["BEAM_RUNNER"] = "DataflowRunner"
os.environ["TRAINING_RUNNER"] = "vertex"
os.environ["TFX_IMAGE_URI"] = f"gcr.io/{PROJECT}/{DATASET_DISPLAY_NAME}:{VERSION}"
os.environ["ENABLE_CACHE"] = "1"

In [11]:
from src.tfx_pipelines import config
import importlib
importlib.reload(config)

for key, value in config.__dict__.items():
    if key.isupper(): print(f'{key}: {value}')

PROJECT: grandelli-demo-295810
REGION: us-central1
GCS_LOCATION: gs://grandelli-demo-295810-partner-training-2022/chicago-taxi-tips
ARTIFACT_STORE_URI: gs://grandelli-demo-295810-partner-training-2022/chicago-taxi-tips/tfx_artifacts
MODEL_REGISTRY_URI: gs://grandelli-demo-295810-partner-training-2022/chicago-taxi-tips/e2e_tests/model_registry
DATASET_DISPLAY_NAME: chicago-taxi-tips
MODEL_DISPLAY_NAME: chicago-taxi-tips-classifier-v01
PIPELINE_NAME: chicago-taxi-tips-classifier-v01-train-pipeline
ML_USE_COLUMN: ml_use
EXCLUDE_COLUMNS: trip_start_timestamp
TRAIN_LIMIT: 85000
TEST_LIMIT: 15000
SERVE_LIMIT: 0
NUM_TRAIN_SPLITS: 4
NUM_EVAL_SPLITS: 1
ACCURACY_THRESHOLD: 0.1
USE_KFP_SA: False
TFX_IMAGE_URI: gcr.io/grandelli-demo-295810/chicago-taxi-tips:v01
BEAM_RUNNER: DataflowRunner
BEAM_DIRECT_PIPELINE_ARGS: ['--project=grandelli-demo-295810', '--temp_location=gs://grandelli-demo-295810-partner-training-2022/chicago-taxi-tips/temp']
BEAM_DATAFLOW_PIPELINE_ARGS: ['--project=grandelli-demo-29

### Build the ML container image

This is the `TFX` runtime environment for the training pipeline steps.

In [12]:
!echo $TFX_IMAGE_URI

gcr.io/grandelli-demo-295810/chicago-taxi-tips:v01


In [13]:
!gcloud builds submit --tag $TFX_IMAGE_URI . --timeout=15m --machine-type=e2-highcpu-8

Creating temporary tarball archive of 59 file(s) totalling 1.6 MiB before compression.
Some files were not included in the source upload.

Check the gcloud log [/home/jupyter/.config/gcloud/logs/2022.03.30/14.52.15.947140.log] to see which files and the contents of the
default gcloudignore file used (see `$ gcloud topic gcloudignore` to learn
more).

Uploading tarball of [.] to [gs://grandelli-demo-295810_cloudbuild/source/1648651937.232844-62032b8b010940f09a1a067690a2ecca.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/grandelli-demo-295810/locations/global/builds/40669306-866f-4f2d-ab9b-85589edef8f9].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/40669306-866f-4f2d-ab9b-85589edef8f9?project=155283586619].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "40669306-866f-4f2d-ab9b-85589edef8f9"

FETCHSOURCE
Fetching storage object: gs://grandelli-demo-295810_cloudbuild/source/1648651937.232844-62032b

### Compile pipeline

In [15]:
from src.tfx_pipelines import runner

pipeline_definition_file = f'{config.PIPELINE_NAME}.json'
pipeline_definition = runner.compile_training_pipeline(pipeline_definition_file)

In [16]:
PIPELINES_STORE = f"gs://{BUCKET}/{DATASET_DISPLAY_NAME}/compiled_pipelines/"
!gsutil cp {pipeline_definition_file} {PIPELINES_STORE}

Copying file://chicago-taxi-tips-classifier-v01-train-pipeline.json [Content-Type=application/json]...
/ [1 files][ 26.6 KiB/ 26.6 KiB]                                                
Operation completed over 1 objects/26.6 KiB.                                     


### Submit run to Vertex Pipelines

In [17]:
from kfp.v2.google.client import AIPlatformClient

pipeline_client = AIPlatformClient(
    project_id=PROJECT, region=REGION)
                 
job = pipeline_client.create_run_from_job_spec(
    job_spec_path=pipeline_definition_file,
    parameter_values={
        'learning_rate': 0.003,
        'batch_size': 512,
        'hidden_units': '128,128',
        'num_epochs': 30,
    }
)

/home/jupyter/.local/lib/python3.7/site-packages/kfp/v2/google/client/client.py:173: FutureWarning: AIPlatformClient will be deprecated in v2.0.0. Please use PipelineJob https://googleapis.dev/python/aiplatform/latest/_modules/google/cloud/aiplatform/pipeline_jobs.html in Vertex SDK. Install the SDK using "pip install google-cloud-aiplatform"
  category=FutureWarning,


### Extracting pipeline runs metadata

In [18]:
from google.cloud import aiplatform as vertex_ai

pipeline_df = vertex_ai.get_pipeline_df(PIPELINE_NAME)
pipeline_df = pipeline_df[pipeline_df.pipeline_name == PIPELINE_NAME]
pipeline_df.T

E0330 17:06:28.880802721       1 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0330 17:06:30.743903842       1 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0330 17:06:32.576653479       1 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


,0,1,2
pipeline_name,chicago-taxi-tips-classifier-v01-train-pipeline,chicago-taxi-tips-classifier-v01-train-pipeline,chicago-taxi-tips-classifier-v01-train-pipeline
run_name,chicago-taxi-tips-classifier-v01-train-pipelin...,chicago-taxi-tips-classifier-v01-train-pipelin...,chicago-taxi-tips-classifier-v01-train-pipelin...
param.input:hidden_units,"128,128","256,126","128,128"
param.input:num_epochs,30,7,30
param.input:batch_size,512,512,512
param.input:learning_rate,0.003,0.0015,0.003


## 3. Execute the pipeline deployment CI/CD steps in Cloud Build

The CI/CD routine is defined in the [pipeline-deployment.yaml](pipeline-deployment.yaml) file, and consists of the following steps:
1. Clone the repository to the build environment.
2. Run unit tests.
3. Run a local e2e test of the pipeline.
4. Build the ML container image for pipeline steps.
5. Compile the pipeline.
6. Upload the pipeline to Cloud Storage.

### Build CI/CD container Image for Cloud Build

This is the runtime environment where the steps of testing and deploying the pipeline will be executed.

In [19]:
!echo $CICD_IMAGE_URI

gcr.io/grandelli-demo-295810/cicd:latest


In [20]:
!gcloud builds submit --tag $CICD_IMAGE_URI build/. --timeout=15m --machine-type=e2-highcpu-8

Creating temporary tarball archive of 5 file(s) totalling 11.2 KiB before compression.
Uploading tarball of [build/.] to [gs://grandelli-demo-295810_cloudbuild/source/1648660042.412306-fc957ae3c7f148d3ba43f9b975d14364.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/grandelli-demo-295810/locations/global/builds/f128fce0-bcfb-4d87-ae0b-eb72bb4d00cb].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/f128fce0-bcfb-4d87-ae0b-eb72bb4d00cb?project=155283586619].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "f128fce0-bcfb-4d87-ae0b-eb72bb4d00cb"

FETCHSOURCE
Fetching storage object: gs://grandelli-demo-295810_cloudbuild/source/1648660042.412306-fc957ae3c7f148d3ba43f9b975d14364.tgz#1648660042653301
Copying gs://grandelli-demo-295810_cloudbuild/source/1648660042.412306-fc957ae3c7f148d3ba43f9b975d14364.tgz#1648660042653301...
/ [1 files][  2.8 KiB/  2.8 KiB]                                                
Ope

### Run CI/CD from pipeline deployment using Cloud Build

In [21]:
REPO_URL = "https://github.com/GoogleCloudPlatform/mlops-with-vertex-ai.git" # Change to your github repo.
BRANCH = "main"

GCS_LOCATION = f"gs://{BUCKET}/{DATASET_DISPLAY_NAME}/"
TEST_GCS_LOCATION = f"gs://{BUCKET}/{DATASET_DISPLAY_NAME}/e2e_tests"
CI_TRAIN_LIMIT = 1000
CI_TEST_LIMIT = 100
CI_UPLOAD_MODEL = 0
CI_ACCURACY_THRESHOLD = 0.1
BEAM_RUNNER = "DataflowRunner"
TRAINING_RUNNER = "vertex"
VERSION = 'tfx-1.2'
PIPELINE_NAME = f'{MODEL_DISPLAY_NAME}-train-pipeline'
PIPELINES_STORE = os.path.join(GCS_LOCATION, "compiled_pipelines")

TFX_IMAGE_URI = f"gcr.io/{PROJECT}/{DATASET_DISPLAY_NAME}:{VERSION}"

SUBSTITUTIONS=f"""\
_REPO_URL='{REPO_URL}',\
_BRANCH={BRANCH},\
_CICD_IMAGE_URI={CICD_IMAGE_URI},\
_PROJECT={PROJECT},\
_REGION={REGION},\
_GCS_LOCATION={GCS_LOCATION},\
_TEST_GCS_LOCATION={TEST_GCS_LOCATION},\
_BQ_LOCATION={BQ_LOCATION},\
_BQ_DATASET_NAME={BQ_DATASET_NAME},\
_BQ_TABLE_NAME={BQ_TABLE_NAME},\
_DATASET_DISPLAY_NAME={DATASET_DISPLAY_NAME},\
_MODEL_DISPLAY_NAME={MODEL_DISPLAY_NAME},\
_CI_TRAIN_LIMIT={CI_TRAIN_LIMIT},\
_CI_TEST_LIMIT={CI_TEST_LIMIT},\
_CI_UPLOAD_MODEL={CI_UPLOAD_MODEL},\
_CI_ACCURACY_THRESHOLD={CI_ACCURACY_THRESHOLD},\
_BEAM_RUNNER={BEAM_RUNNER},\
_TRAINING_RUNNER={TRAINING_RUNNER},\
_TFX_IMAGE_URI={TFX_IMAGE_URI},\
_PIPELINE_NAME={PIPELINE_NAME},\
_PIPELINES_STORE={PIPELINES_STORE}\
"""

!echo $SUBSTITUTIONS

_REPO_URL=https://github.com/GoogleCloudPlatform/mlops-with-vertex-ai.git,_BRANCH=main,_CICD_IMAGE_URI=gcr.io/grandelli-demo-295810/cicd:latest,_PROJECT=grandelli-demo-295810,_REGION=us-central1,_GCS_LOCATION=gs://grandelli-demo-295810-partner-training-2022/chicago-taxi-tips/,_TEST_GCS_LOCATION=gs://grandelli-demo-295810-partner-training-2022/chicago-taxi-tips/e2e_tests,_BQ_LOCATION=US,_BQ_DATASET_NAME=partner_training,_BQ_TABLE_NAME=chicago_taxitrips_prep,_DATASET_DISPLAY_NAME=chicago-taxi-tips,_MODEL_DISPLAY_NAME=chicago-taxi-tips-classifier-v01,_CI_TRAIN_LIMIT=1000,_CI_TEST_LIMIT=100,_CI_UPLOAD_MODEL=0,_CI_ACCURACY_THRESHOLD=0.1,_BEAM_RUNNER=DataflowRunner,_TRAINING_RUNNER=vertex,_TFX_IMAGE_URI=gcr.io/grandelli-demo-295810/chicago-taxi-tips:tfx-1.2,_PIPELINE_NAME=chicago-taxi-tips-classifier-v01-train-pipeline,_PIPELINES_STORE=gs://grandelli-demo-295810-partner-training-2022/chicago-taxi-tips/compiled_pipelines


In [22]:
!gcloud builds submit --no-source --timeout=60m --config build/pipeline-deployment.yaml --substitutions {SUBSTITUTIONS} --machine-type=e2-highcpu-8

Created [https://cloudbuild.googleapis.com/v1/projects/grandelli-demo-295810/locations/global/builds/13834e3f-3ac3-4096-9e01-cd058a1a30d0].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/13834e3f-3ac3-4096-9e01-cd058a1a30d0?project=155283586619].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "13834e3f-3ac3-4096-9e01-cd058a1a30d0"

FETCHSOURCE
BUILD
Starting Step #0 - "Clone Repository"
Step #0 - "Clone Repository": Already have image (with digest): gcr.io/cloud-builders/git
Step #0 - "Clone Repository": Cloning into 'mlops-with-vertex-ai'...
Step #0 - "Clone Repository": POST git-upload-pack (352 bytes)
Step #0 - "Clone Repository": POST git-upload-pack (194 bytes)
Finished Step #0 - "Clone Repository"
Starting Step #1 - "Unit Test Datasource Utils"
Starting Step #2 - "Unit Test Model"
Step #1 - "Unit Test Datasource Utils": Pulling image: gcr.io/grandelli-demo-295810/cicd:latest
Step #2 - "Unit Test Model": 